In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import altair as alt

In [8]:
df = pd.read_csv('linkedin_data_sample.csv')

In [9]:
def generate_meta(df):
    """ generates meta data for a df """

    df_meta = pd.DataFrame()
    df_meta['datatype'] = df.dtypes
    df_meta['total_values'] = [df[col].size - df[col].isna().sum() for col in df.columns]
    df_meta['null_values'] = [df[col].isna().sum() for col in df.columns]
    df_meta['distinct_values'] = [len(set(df[col])) for col in df.columns]

    return df_meta

# keep only usd
df = df[df.currency=='USD']
df = df[df.normalized_salary >= 15000]

# explore data
df_meta = generate_meta(df)

# drop columns with all null, single value, or untelligible values
drop_cols = list(df_meta[(df_meta.distinct_values==1) | (df_meta.null_values == len(df))].index)
drop_cols.extend(['expiry', 'listed_time'])
drop_cols.extend(['expiry', 
            'listed_time', 
            'application_type',
            'application_url',
            'company_id',
            'posting_domain',
            'skills_desc'])

# drop rows without a fips (location)
df.dropna(subset=['fips'], inplace=True)
df['fips_state'] = [int(str(string)[:-5]) for string in df.fips]
            
print('dropping these columns:', drop_cols)
df.drop(drop_cols, axis=1, inplace=True)
print(df.shape)

generate_meta(df)

dropping these columns: ['pay_period', 'med_salary', 'sponsored', 'work_type', 'currency', 'compensation_type', 'expiry', 'listed_time', 'expiry', 'listed_time', 'application_type', 'application_url', 'company_id', 'posting_domain', 'skills_desc']
(9630, 17)


datatype  total_values  null_values  \
Unnamed: 0                    int64          9630            0   
job_id                        int64          9630            0   
company_name                 object          9630            0   
title                        object          9630            0   
description                  object          9630            0   
max_salary                  float64          9630            0   
location                     object          9630            0   
views                       float64          9630            0   
min_salary                  float64          9630            0   
applies                     float64          2476         7154   
remote_allowed              float64           698         8932   
job_posting_url              object          9630            0   
formatted_experience_level   object          9630            0   
normalized_salary           float64          9630            0   
zip_code                    float64          9630            0   
fips                        float64          9630            0   
fips_state                    int64          9630            0   

                            distinct_values  
Unnamed: 0                             9630  
job_id                                 9630  
company_name                           3524  
title                                  7248  
description                            9140  
max_salary                             2041  
location                               1738  
views                                   207  
min_salary                             1864  
applies                                7243  
remote_allowed                         8933  
job_posting_url                        9630  
formatted_experience_level                6  
normalized_salary                      2431  
zip_code                               1657  
fips                                    806  
fips_state                               51

In [10]:
job_industries = pd.read_csv('job_industries.csv')
industries = pd.read_csv('industries.csv')
df_jobs = pd.merge(job_industries, industries, on='industry_id', how='inner')
df_jobs = df_jobs.groupby('job_id')['industry_name'].agg(lambda x: list(x.unique())).reset_index()
df_jobs['first_industry'] = [ind_list[0] for ind_list in df_jobs.industry_name]

job_skills = pd.read_csv('job_skills.csv')
skills = pd.read_csv('skills.csv')
job_skills = pd.merge(job_skills, skills, on='skill_abr', how='inner')
job_skills = job_skills.groupby('job_id')['skill_name'].agg(lambda x: list(x.unique())).reset_index()

df_jobs = pd.merge(df_jobs, job_skills, on='job_id', how='inner')
df = pd.merge(df_jobs, df, on='job_id')

import geopandas 
state = geopandas.read_file('ne_110m_admin_1_states_provinces')
state['fips_state'] = [int(string[2:]) for string in state.fips]
region_state = state[['fips_state','region']]
df = pd.merge(df, region_state, on='fips_state', how='left')

industry_levels = pd.read_csv('industry_level_mapping.csv')
df = pd.merge(df, industry_levels, left_on='first_industry', right_on='Industry')

In [11]:
column_order = ['job_id', 
'company_name', 
'title', 
'location',
'region',
'Sector',
'first_industry', 
'industry_name', 
'skill_name',
'formatted_experience_level', 
'description',
'normalized_salary', 
'max_salary', 
'min_salary',
'views', 
'applies', 
'remote_allowed', 
'zip_code',
'fips',
'fips_state',
'job_posting_url']

df = df[column_order]
print(df.shape)
df.head()

(9545, 21)


job_id          company_name                              title  \
0  2989631782          ActOne Group  Administrative Assistant - CONCUR   
1  3177010992  ABC Farigua Division    Customer Service Representative   
2  3619548798       Dexterity, Inc.         Senior Mechanical Engineer   
3  3681437990             Ascendion       Quality Assurance Specialist   
4  3803052628        Insight Global           Flight Software Engineer   

              location     region                      Sector  \
0         New York, NY  Northeast          Financial Services   
1  Greater Orlando, FL      South          Financial Services   
2     Redwood City, CA       West    Technology & IT Services   
3          Seattle, WA       West     Consumer Goods & Retail   
4          Webster, TX      South  Manufacturing & Industrial   

                      first_industry  \
0                 Financial Services   
1  Insurance Agencies and Brokerages   
2    Computer Hardware Manufacturing   
3                             Retail   
4    Defense and Space Manufacturing   

                                       industry_name  \
0                               [Financial Services]   
1  [Insurance Agencies and Brokerages, Food and B...   
2  [Computer Hardware Manufacturing, Software Dev...   
3  [Retail, Food and Beverage Services, Food and ...   
4                  [Defense and Space Manufacturing]   

                            skill_name formatted_experience_level  ...  \
0                     [Administrative]                  Associate  ...   
1            [Customer Service, Sales]                Entry level  ...   
2                        [Engineering]           Mid-Senior level  ...   
3  [Administrative, Quality Assurance]                  Associate  ...   
4                        [Engineering]           Mid-Senior level  ...   

  normalized_salary  max_salary  min_salary  views  applies  remote_allowed  \
0           82500.0     90000.0     75000.0    1.0      NaN             NaN   
1           90000.0    105000.0     75000.0    6.0      NaN             NaN   
2          170000.0    190000.0    150000.0    3.0      NaN             NaN   
3           66000.0     72000.0     60000.0    8.0      NaN             NaN   
4          155000.0    170000.0    140000.0   36.0      7.0             NaN   

   zip_code     fips  fips_state  \
0   10001.0  36061.0          36   
1   32801.0  12095.0          12   
2   94061.0   6081.0           6   
3   98101.0  53033.0          53   
4   77598.0  48201.0          48   

                                     job_posting_url  
0  https://www.linkedin.com/jobs/view/2989631782/...  
1  https://www.linkedin.com/jobs/view/3177010992/...  
2  https://www.linkedin.com/jobs/view/3619548798/...  
3  https://www.linkedin.com/jobs/view/3681437990/...  
4  https://www.linkedin.com/jobs/view/3803052628/...  

[5 rows x 21 columns]

In [12]:
df.to_csv('linkedin_df.csv')